In [1]:
import numpy as np
import keras
from keras.models import load_model
from game2048.game import Game
from game2048.agents import ExpectiMaxAgent
from game2048.expectimax import board_to_move

from sklearn.preprocessing import OneHotEncoder
import numpy as np


import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"



n_epochs = 5
batch_size = 300
board_dimension = 3


model_256_path = '/home/srtp/ML/2048-api-master/2048-api-master/maybe_339/train_256.h5'
model_512_path = '/home/srtp/ML/2048-api-master/2048-api-master/maybe_339/train_512.h5'
model_1024_path = '/home/srtp/ML/2048-api-master/2048-api-master/maybe_339/train_1024.h5'
model_2048_path = '/home/srtp/ML/2048-api-master/2048-api-master/maybe_339/train_2048.h5'





ohe = OneHotEncoder()
ohe.fit([ [0], [2], [4], [8], [16], [32], [64], [128], [256], [512], [1024], [2048]])

def board_transform(board_to_transe, gamesize=4):
    """transform the board by one-hot encoding"""
    if board_dimension == 3:
        inputboard = np.zeros(( 16, 12))
        for p in range(4):
            for q in range(4):
                num = board_to_transe[p, q]
                if num == 0:
                    inputboard[ 4*p+q, 0] = 1
                else:
                    inputboard[ 4*p+q, int(np.log2(num))] = 1 
    elif board_dimension==4:
        inputboard = np.zeros(( 4, 4, 12))
        for p in range(4):
            for q in range(4):
                num = board_to_transe[p, q]
                if num == 0:
                    inputboard[ p, q, 0] = 1
                else:
                    inputboard[ p, q, int(np.log2(num))] = 1
    else:
        print('OH NO! you data denmension is not ture!')

    return inputboard



class ModelWrapper:
    def __init__(self, capacity, play_times_until_output=50):
        self.model_256 = load_model(model_256_path)
        self.model_512 = load_model(model_512_path)
        self.model_1024 = load_model(model_1024_path)
        self.model_2048 = load_model(model_2048_path)
        
        self.boards_256 = []
        self.boards_512 = []
        self.boards_1024 = []
        self.boards_2048 = []
        
        self.directions_256 = []
        self.directions_512 = []
        self.directions_1024 = []
        self.directions_2048 = []
        
        self.capacity = capacity
        self.play_times_until_output = play_times_until_output
        
        self.scores = []
        
    def predict(self, ohe_board, score):
        #ohe_board = board_transform(board)
        
        if score < 256:
            ohe_direction = self.model_256.predict(np.expand_dims(ohe_board,axis = 0))
            return np.argmax(ohe_direction[0])
        elif score< 512:
            ohe_direction = self.model_512.predict(np.expand_dims(ohe_board,axis = 0))
            return np.argmax(ohe_direction[0])
        elif score< 1024:
            ohe_direction = self.model_1024.predict(np.expand_dims(ohe_board,axis = 0))
            return np.argmax(ohe_direction[0])
        elif score<2048:
            ohe_direction = self.model_2048.predict(np.expand_dims(ohe_board,axis = 0))
            return np.argmax(ohe_direction[0])        
        
    def move(self, game):
        now_board = board_transform(game.board)
        suggest = board_to_move(game.board)
        
        if game.score < 256:
            self.boards_256.append(now_board)
            self.directions_256.append(suggest)
            direction = self.predict(now_board, game.score)
            game.move(direction)
        elif game.score < 512:
            self.boards_512.append(now_board)
            self.directions_512.append(suggest)
            direction = self.predict(now_board, game.score)
            game.move(direction)
        elif game.score < 1024:
            self.boards_1024.append(now_board)
            self.directions_1024.append(suggest)
            direction = self.predict(now_board, game.score)
            game.move(direction)
        else:
            self.boards_2048.append(now_board)
            self.directions_2048.append(suggest)
            direction = self.predict(now_board, game.score)
            game.move(direction)

    def show_scores(self):
        #show the scores every play_times
        if len(self.scores) >= self.play_times_until_output:
            print("======================================")
            print("score is ",self.scores)
            print("Average scores: @%s times"%self.play_times_until_output, sum(self.scores) / len(self.scores))
            print("======================================")
            self.scores = []
               
    def judge_end(self, game):
        if game.end>0:
            self.scores.append(game.score)
            print ('boards_256 = ', len(self.boards_256))
            print ('boards_512 = ', len(self.boards_512))
            print ('boards_1024 = ', len(self.boards_1024))
            print ('boards_2048 = ', len(self.boards_2048))
            self.show_scores()
            return True
        else:
            return False
        
    def train(self):
        if len(self.boards_256) >= self.capacity:
            self.boards_256 = np.array(self.boards_256)
            self.directions_256 = np.array(self.directions_256)
            self.directions_256 = keras.utils.to_categorical(self.directions_256, num_classes=4)
            print("training on model_256")
            self.model_256.fit(self.boards_256, self.directions_256, epochs=n_epochs, batch_size=batch_size, validation_split=0.05)
            self.model_256.save(model_256_path)
            self.boards_256 = []
            self.directions_256 = []
            
        if len(self.boards_512) >= self.capacity:
            self.boards_512 = np.array(self.boards_512)
            self.directions_512 = np.array(self.directions_512)
            self.directions_512 = keras.utils.to_categorical(self.directions_512, num_classes=4)
            print("training on model_512")
            self.model_512.fit(self.boards_512, self.directions_512, epochs=n_epochs, batch_size=batch_size, validation_split=0.05)
            self.model_512.save(model_512_path)
            self.boards_512 = []
            self.directions_512 = []
            
        if len(self.boards_1024) >= self.capacity:
            self.boards_1024 = np.array(self.boards_1024)
            self.directions_1024 = np.array(self.directions_1024)
            self.directions_1024 = keras.utils.to_categorical(self.directions_1024, num_classes=4)
            print("training on model_1024")
            self.model_1024.fit(self.boards_1024, self.directions_1024, epochs=n_epochs, batch_size=batch_size, validation_split=0.05)
            self.model_1024.save(model_1024_path)
            self.boards_1024 = []
            self.directions_1024 = []
        
        if len(self.boards_2048) >= self.capacity:
            self.boards_2048 = np.array(self.boards_2048)
            self.directions_2048 = np.array(self.directions_2048)
            self.directions_2048 = keras.utils.to_categorical(self.directions_2048, num_classes=4)
            print("training on model_2048")
            self.model_2048.fit(self.boards_2048, self.directions_2048, epochs=n_epochs, batch_size=batch_size, validation_split=0.05)
            self.model_2048.save(model_2048_path)
            self.boards_2048 = []
            self.directions_2048 = []

        
                
zit_model = ModelWrapper(capacity=2000,play_times_until_output=50)
for i in range(100000000):
    print('------')
    print('play_times = ', i)
    game = Game(size=4)
    while True:           
        if zit_model.judge_end(game):
            break
        else:
            zit_model.move(game)
            zit_model.train()
print('end')

Using TensorFlow backend.


Loaded expectmax lib for 2048: /home/srtp/ML/2048-api-master/2048-api-master/game2048/expectimax/bin/2048.so
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
------
play_times =  0
boards_256 =  103
boards_512 =  69
boards_1024 =  0
boards_2048 =  0
------
play_times =  1
boards_256 =  160
boards_512 =  69
boards_1024 =  0
boards_2048 =  0
------
play_times =  2
boards_256 =  272
boards_512 =  69
boards_1024 =  0
boards_2048 =  0
------
play_times =  3
boards_256 =  359
boards_512 =  69
boards_1024 =  0
boards_2048 =  0
------
play_times =  4
boards_256 =  474
boards_512 =  149
boards_1024 =  0
boards_2048 =  0
------
play_times =  5
boards_256 =  582
boards_512 =  235
boards_1024 =  0
boards_2048 =  0
------
play_times =  6
boards_256 =  691
boards_512 =  270
boards_1024 =  0
boards_2048 =  0
------
play_times =  7
boards_256 =  815
boards_512 =  328
boards_1024 =  0
boards_2048 =  0
-----

1900/1900 [==============================] - 2s 962us/step - loss: 1.1533 - acc: 0.4647 - val_loss: 1.2084 - val_acc: 0.5000
Epoch 2/5
1900/1900 [==============================] - 2s 978us/step - loss: 1.1576 - acc: 0.4547 - val_loss: 1.2167 - val_acc: 0.5000
Epoch 3/5
1900/1900 [==============================] - 2s 979us/step - loss: 1.1534 - acc: 0.4589 - val_loss: 1.2150 - val_acc: 0.5100
Epoch 4/5
1900/1900 [==============================] - 2s 978us/step - loss: 1.1235 - acc: 0.4863 - val_loss: 1.2290 - val_acc: 0.5000
Epoch 5/5
1900/1900 [==============================] - 2s 984us/step - loss: 1.1151 - acc: 0.4789 - val_loss: 1.2434 - val_acc: 0.5000
boards_256 =  26
boards_512 =  1468
boards_1024 =  383
boards_2048 =  0
------
play_times =  59
boards_256 =  124
boards_512 =  1560
boards_1024 =  383
boards_2048 =  0
------
play_times =  60
boards_256 =  246
boards_512 =  1607
boards_1024 =  383
boards_2048 =  0
------
play_times =  61
boards_256 =  311
boards_512 =  1607
boards_1

boards_256 =  1103
boards_512 =  696
boards_1024 =  618
boards_2048 =  0
------
play_times =  110
boards_256 =  1236
boards_512 =  696
boards_1024 =  618
boards_2048 =  0
------
play_times =  111
boards_256 =  1315
boards_512 =  696
boards_1024 =  618
boards_2048 =  0
------
play_times =  112
boards_256 =  1431
boards_512 =  773
boards_1024 =  618
boards_2048 =  0
------
play_times =  113
boards_256 =  1456
boards_512 =  773
boards_1024 =  618
boards_2048 =  0
------
play_times =  114
boards_256 =  1607
boards_512 =  901
boards_1024 =  618
boards_2048 =  0
------
play_times =  115
boards_256 =  1709
boards_512 =  987
boards_1024 =  618
boards_2048 =  0
------
play_times =  116
boards_256 =  1780
boards_512 =  987
boards_1024 =  618
boards_2048 =  0
------
play_times =  117
boards_256 =  1902
boards_512 =  987
boards_1024 =  618
boards_2048 =  0
------
play_times =  118
training on model_256
Train on 1900 samples, validate on 100 samples
Epoch 1/5
1900/1900 [============================

Epoch 3/5
1900/1900 [==============================] - 2s 957us/step - loss: 1.0534 - acc: 0.5142 - val_loss: 1.1138 - val_acc: 0.4700
Epoch 4/5
1900/1900 [==============================] - 2s 953us/step - loss: 0.9920 - acc: 0.5574 - val_loss: 1.1129 - val_acc: 0.4700
Epoch 5/5
1900/1900 [==============================] - 2s 961us/step - loss: 0.9039 - acc: 0.6147 - val_loss: 1.1257 - val_acc: 0.4700
boards_256 =  394
boards_512 =  87
boards_1024 =  694
boards_2048 =  0
------
play_times =  164
boards_256 =  493
boards_512 =  178
boards_1024 =  694
boards_2048 =  0
------
play_times =  165
boards_256 =  617
boards_512 =  193
boards_1024 =  694
boards_2048 =  0
------
play_times =  166
boards_256 =  726
boards_512 =  193
boards_1024 =  694
boards_2048 =  0
------
play_times =  167
boards_256 =  857
boards_512 =  193
boards_1024 =  694
boards_2048 =  0
------
play_times =  168
boards_256 =  902
boards_512 =  193
boards_1024 =  694
boards_2048 =  0
------
play_times =  169
boards_256 =  

...

boards_256 =  853
boards_512 =  316
boards_1024 =  384
boards_2048 =  848
------
play_times =  12001
boards_256 =  956
boards_512 =  427
boards_1024 =  466
boards_2048 =  848
------
play_times =  12002
boards_256 =  1015
boards_512 =  427
boards_1024 =  466
boards_2048 =  848
------
play_times =  12003
boards_256 =  1167
boards_512 =  485
boards_1024 =  466
boards_2048 =  848
------
play_times =  12004
boards_256 =  1278
boards_512 =  504
boards_1024 =  466
boards_2048 =  848
------
play_times =  12005
boards_256 =  1391
boards_512 =  532
boards_1024 =  466
boards_2048 =  848
------
play_times =  12006
boards_256 =  1521
boards_512 =  594
boards_1024 =  571
boards_2048 =  848
------
play_times =  12007
boards_256 =  1672
boards_512 =  649
boards_1024 =  622
boards_2048 =  848
------
play_times =  12008
boards_256 =  1783
boards_512 =  768
boards_1024 =  622
boards_2048 =  848
------
play_times =  12009
boards_256 =  1874
boards_512 =  895
boards_1024 =  775
boards_2048 =  848
------
pl

boards_256 =  500
boards_512 =  1604
boards_1024 =  1796
boards_2048 =  1001
------
play_times =  12051
boards_256 =  610
boards_512 =  1695
boards_1024 =  1796
boards_2048 =  1001
------
play_times =  12052
boards_256 =  723
boards_512 =  1734
boards_1024 =  1796
boards_2048 =  1001
------
play_times =  12053
boards_256 =  832
boards_512 =  1734
boards_1024 =  1796
boards_2048 =  1001
------
play_times =  12054
boards_256 =  931
boards_512 =  1876
boards_1024 =  1859
boards_2048 =  1001
------
play_times =  12055
boards_256 =  1031
boards_512 =  1876
boards_1024 =  1859
boards_2048 =  1001
------
play_times =  12056
boards_256 =  1159
boards_512 =  1956
boards_1024 =  1903
boards_2048 =  1001
------
play_times =  12057
training on model_512
Train on 1900 samples, validate on 100 samples
Epoch 1/5
1900/1900 [==============================] - 2s 936us/step - loss: 1.0202 - acc: 0.5142 - val_loss: 1.0731 - val_acc: 0.4800
Epoch 2/5
1900/1900 [==============================] - 2s 943us/st

boards_256 =  1515
boards_512 =  639
boards_1024 =  1300
boards_2048 =  1001
------
play_times =  12095
boards_256 =  1626
boards_512 =  673
boards_1024 =  1300
boards_2048 =  1001
------
play_times =  12096
boards_256 =  1742
boards_512 =  753
boards_1024 =  1300
boards_2048 =  1001
------
play_times =  12097
boards_256 =  1843
boards_512 =  832
boards_1024 =  1300
boards_2048 =  1001
------
play_times =  12098
boards_256 =  1968
boards_512 =  926
boards_1024 =  1300
boards_2048 =  1001
------
play_times =  12099
training on model_256
Train on 1900 samples, validate on 100 samples
Epoch 1/5
1900/1900 [==============================] - 2s 960us/step - loss: 0.9424 - acc: 0.5379 - val_loss: 1.0665 - val_acc: 0.5500
Epoch 2/5
1900/1900 [==============================] - 2s 961us/step - loss: 0.9406 - acc: 0.5489 - val_loss: 1.0609 - val_acc: 0.5400
Epoch 3/5
1900/1900 [==============================] - 2s 959us/step - loss: 0.9259 - acc: 0.5679 - val_loss: 1.0585 - val_acc: 0.5000
Epoch 

1900/1900 [==============================] - 2s 955us/step - loss: 0.8842 - acc: 0.5963 - val_loss: 0.8939 - val_acc: 0.4800
Epoch 5/5
1900/1900 [==============================] - 2s 990us/step - loss: 0.8529 - acc: 0.5963 - val_loss: 0.8896 - val_acc: 0.5000
boards_256 =  77
boards_512 =  1354
boards_1024 =  516
boards_2048 =  1001
------
play_times =  12137
boards_256 =  202
boards_512 =  1442
boards_1024 =  642
boards_2048 =  1001
------
play_times =  12138
boards_256 =  353
boards_512 =  1469
boards_1024 =  642
boards_2048 =  1001
------
play_times =  12139
boards_256 =  469
boards_512 =  1527
boards_1024 =  642
boards_2048 =  1001
------
play_times =  12140
boards_256 =  580
boards_512 =  1647
boards_1024 =  732
boards_2048 =  1001
------
play_times =  12141
boards_256 =  717
boards_512 =  1725
boards_1024 =  781
boards_2048 =  1001
------
play_times =  12142
boards_256 =  837
boards_512 =  1791
boards_1024 =  781
boards_2048 =  1001
------
play_times =  12143
boards_256 =  939
bo

KeyboardInterrupt: 